In [36]:
# NOTE: https://www.unb.ca/cic/datasets/nsl.html
# started w/ this http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Normalizer

data = pd.read_csv("corrected")

In [17]:
data.shape

(311028, 42)

In [18]:
data.head()

,0,udp,private,SF,105,146,0.1,0.2,0.3,0.4,...,254,1.00.1,0.01,0.00.6,0.00.7,0.00.8,0.00.9,0.00.10,0.00.11,normal.
0,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,snmpgetattack.
3,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack.
4,0,udp,private,SF,105,146,0,0,0,0,...,255,1.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack.


In [19]:
len(data.columns)

42

In [51]:
field = pd.read_csv('NSL-KDD/Field Names.csv', header=None)
field.columns = ["Field", "Type"]
field.transpose()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
Field,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
Type,continuous,symbolic,symbolic,symbolic,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous


In [56]:
# Establish test and train data
traindata = pd.read_csv('NSL-KDD/KDDTrain+.csv', header=None)
traindata[41].unique()
# testdata = pd.read_csv('NSL-KDD/KDDTest+.txt', header=None)

array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back', 'guess_passwd',
       'ftp_write', 'multihop', 'rootkit', 'buffer_overflow', 'imap',
       'warezmaster', 'phf', 'land', 'loadmodule', 'spy', 'perl'], dtype=object)

In [69]:
labels = pd.read_csv("NSL-KDD/Attack Types.csv", header=None)
for val in traindata[41].unique():
    found = False
    for l in labels.iloc[:,0]:
        if val == l:
            found = True
    if not found:
        print(val)

In [70]:
traindata.groupby(41).size()

41
back                 956
buffer_overflow       30
ftp_write              8
guess_passwd          53
imap                  11
ipsweep             3599
land                  18
loadmodule             9
multihop               7
neptune            41214
nmap                1493
normal             67343
perl                   3
phf                    4
pod                  201
portsweep           2931
rootkit               10
satan               3633
smurf               2646
spy                    2
teardrop             892
warezclient          890
warezmaster           20
dtype: int64

## Naive Bayes

In [42]:
traindata = pd.read_csv('NSL-KDD/KDDTrain+.csv', header=None)
testdata = pd.read_csv('NSL-KDD/KDDTest+.csv', header=None)

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

In [43]:
scaler = Normalizer().fit(X)

ValueError: could not convert string to float: 'normal'

In [ ]:
trainX = scaler.transform(X)

In [33]:
model.fit(traindata, labels)
print(model)

ValueError: could not convert string to float: 'normal'

In [ ]:
expected = testlabel
predicted = model.predict(testdata)